<a href="https://colab.research.google.com/github/albertorock82/coursera_frontend/blob/main/obtenerGruposdeKW_frecuencia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --quiet gspread pandas unidecode nltk


In [ ]:
# Paso 1: Instalar dependencias
!pip install --quiet gspread pandas unidecode nltk

import pandas as pd
import gspread
from unidecode import unidecode
import nltk
from nltk.corpus import stopwords
import string
import re

# Descargar stopwords
nltk.download('stopwords')

from google.colab import auth
from google.auth.transport.requests import Request
from google.oauth2.service_account import Credentials

# --- PARAMETROS (MODIFICA ESTO) ---
NOMBRE_ARCHIVO_JSON = '/content/ambient-antenna-440101-u4-d9758bb09208.json'  # Sube tu clave JSON y pon aquí el nombre
NOMBRE_LIBRO = 'KeywordDepliadorKS_serp'
NOMBRE_HOJA = 'Sheet1'  # hoja donde están las keywords en columna A
NOMBRE_HOJA_RESULTADO = 'grupos'

# --- AUTENTICACIÓN ---
scope = ["https://spreadsheets.google.com/feeds",'https://www.googleapis.com/auth/spreadsheets',
         "https://www.googleapis.com/auth/drive.file","https://www.googleapis.com/auth/drive"]

credentials = Credentials.from_service_account_file(NOMBRE_ARCHIVO_JSON, scopes=scope)
gc = gspread.authorize(credentials)

# --- CARGAR DATOS DEL SHEET ---
sh = gc.open(NOMBRE_LIBRO)
worksheet = sh.worksheet(NOMBRE_HOJA)
keywords = worksheet.col_values(1)  # columna A

# --- LIMPIEZA Y TOKENIZACIÓN ---
# Stopwords en español + extras
stopwords_es = set(stopwords.words('spanish'))
stopwords_extras = set(["como", "es", "de", "los", "las", "para", "el", "la", "por", "con", "un", "una", "al", "en"])
stopwords_all = stopwords_es.union(stopwords_extras)

# Expresión regular para detectar palabras que tienen solo números
regex_solo_numeros = re.compile(r'^\d+$')

# Preprocesar y tokenizar
def limpiar_y_tokenizar(texto):
    texto = texto.lower()
    texto = unidecode(texto)  # elimina tildes
    palabras = texto.translate(str.maketrans('', '', string.punctuation)).split()

    palabras_filtradas = []
    for p in palabras:
        if (
            len(p) >= 3 and                           # Al menos 3 letras
            p not in stopwords_all and               # No esté en stopwords
            not re.match(r'^\d+$', p)                # No sea solo números
        ):
            palabras_filtradas.append(p)

    return palabras_filtradas


# Contar palabras
from collections import Counter
contador = Counter()

for kw in keywords:
    palabras = limpiar_y_tokenizar(kw)
    contador.update(palabras)

# Convertir resultados en DataFrame ordenado
df_resultado = pd.DataFrame(contador.items(), columns=["palabra", "frecuencia"]).sort_values("frecuencia", ascending=False)

# --- CREAR NUEVA HOJA Y CARGAR RESULTADOS ---
# Si la hoja "grupos" ya existe, borrarla
try:
    sh.del_worksheet(sh.worksheet(NOMBRE_HOJA_RESULTADO))
except:
    pass

# Crear nueva hoja y cargar
worksheet_nueva = sh.add_worksheet(title=NOMBRE_HOJA_RESULTADO, rows=len(df_resultado)+1, cols=2)
worksheet_nueva.update([["palabra", "frecuencia"]] + df_resultado.values.tolist())

print("✅ Palabras agrupadas y guardadas en la hoja 'grupos'")


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


✅ Palabras agrupadas y guardadas en la hoja 'grupos'
